# Stereo Vision Pipeline for Disparity Map Estimation

This notebook describes the complete pipeline for computing disparity maps using a stereo camera system.
The goal is to improve the quality and robustness of disparity estimation by refining each processing step,
including camera calibration, image rectification, distortion correction, and post-processing.

The pipeline consists of the following steps:

1. Disparity map computation using StereoSGBM
2. Disparity map computation with hole-filling post-processing
3. Improved camera calibration (estimation of intrinsic parameters and distortion coefficients)
4. Verification and tuning of distortion correction using calibration parameters
5. Image rectification and undistortion
6. Disparity map computation using rectified and undistorted images

Each step is explained below with its purpose and role in the overall pipeline.

## 2.2. Validation of Calibration Parameters Using the Baseline Method

In this step, camera intrinsic parameters and distortion coefficients
are estimated using a standard calibration method without additional filtering.

To evaluate the reliability of these parameters, distortion correction
is applied to the original images using the estimated values.
The checkerboard is then reprojected onto the undistorted images.

This process allows us to assess calibration quality by observing:

- Alignment between projected points and actual checkerboard corners
- Residual distortion near image boundaries
- Visual consistency across different calibration images

This baseline evaluation serves as a reference for later improvements.


In [ ]:
import cv2
import numpy as np
import glob
from google.colab.patches import cv2_imshow
from google.colab import drive


drive.mount('/content/drive')


CHECKERBOARD = (7, 7)
SQUARE_SIZE = 1.41
IMG_DIR = "/content/drive/MyDrive/Amazon project/chess"


objp = np.zeros((CHECKERBOARD[0]*CHECKERBOARD[1],3), np.float32)
objp[:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1,2)
objp *= SQUARE_SIZE

objpoints = []  # 3Dポイント
imgpoints = []  # 2Dポイント

# ===== 画像読み込み =====
image_files = glob.glob(IMG_DIR + "/*.jpg")
if len(image_files) == 0:
    print("画像が見つかりません。パスを確認してください。")

# ===== コーナー検出 =====
for fname in image_files:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)
    if ret:
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)

        objpoints.append(objp)
        imgpoints.append(corners2)

        # # コーナーを描画して確認
        # cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
        # cv2_imshow(img)
    else:
        print(f"コーナー検出失敗: {fname}")

# ===== カメラキャリブレーション =====
ret, K, D, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print("再投影誤差:", ret)
print("カメラ行列:\n", K)
print("歪み係数:\n", D)

D_n= 0.05 * D
# ===== 最適カメラ行列（端の過剰補正を抑える alpha=0） =====
new_K, roi = cv2.getOptimalNewCameraMatrix(K, D_n, gray.shape[::-1], alpha=0)

# ===== 歪み補正 =====
for fname in image_files:
    img = cv2.imread(fname)
    undistorted = cv2.undistort(img, K, D_n, None, new_K)

    # ROIで切り取り（必要に応じて）
    x, y, w_roi, h_roi = roi
    undistorted = undistorted[y:y+h_roi, x:x+w_roi]

    # 表示
    print(f"補正後画像: {fname}")
    cv2_imshow(undistorted)

    # 保存する場合
    save_path = fname.replace("chess", "chess/processed")
    cv2.imwrite(save_path, undistorted)